In [75]:
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

examples = [
    {
        "question": "Top Gun",
        "answer": "Emojis: 🛩️👨‍✈️🔥\nKeywords: Fighter Jets, Brotherhood, Competition"
    },
    {
        "question": "The Godfather",
        "answer": "Emojis: 👨‍👨‍👦🔫🍝\nKeywords: Mafia, Family, Power"
    },
    {
        "question": "ET",
        "answer": "Emojis: 👽🚲🌕\nKeywords: Alien, Friendship, Adventure"
    },
    {
        "question": "what was the movie I asked about first?",
        "answer": "You asked about the Top Gun and I provided Emojis: 🛩️👨‍✈️🔥\nKeywords: Fighter Jets, Brotherhood, Competition"
    }
]


example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Give me 3 emojis and 3 keywords that represent the movie {question}"),
    ("ai", "{answer}")
])

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt, 
    examples = examples
)

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.1,
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationBufferMemory(
    llm=llm, max_token_limit=80,
    memory_key="history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", 
        "You are a movie expert. You always respond with 3 emojis and 3 keywords representing the movie. "
        "If the user asks about previous questions, recall from the conversation history below and answer clearly."
    ),
    MessagesPlaceholder(variable_name="history"),  # 대화 기록 포함
    fewshot_prompt,  # 랜덤 예제 포함
    ("human", "Answer the question. "
        "If the question is a movie title, give 3 emojis and 3 keywords for the movie '{question}'. "
        "If I ask about past questions, answer based on conversation history.")
])


def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt| llm

def invoke_chain(question):
    response = chain.invoke({"question": question})
    memory.save_context(
        {"input": question}, 
        {"output": response.content}
    ) 

In [76]:
invoke_chain("Interstellar")

Emojis: 🚀🌌⏳
Keywords: Space Travel, Time Dilation, Love

In [77]:
invoke_chain("Dune")

Emojis: 🏜️🚀👑
Keywords: Sci-Fi, Desert, Power

In [78]:
invoke_chain("what was the movie I asked about first?")

Interstellar 🚀🌌⏳
Space Travel, Time Dilation, Love